# Puissance 4 Exo

In [19]:
from typing import List, Dict, Tuple
import random
import copy
import time

In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.output_wrapper, .output { height:auto !important; max-height: 200000px; } .output_scroll { box-shadow:none !important; webkit-box-shadow:none !important; }</style>"))

/tmp/ipykernel_105218/2531052898.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Global Functions definition

In [4]:
def print_board(board:List[List[int]]):
    """print the board 

    Args:
        board (List[List[int]]): the game board
    """
    print(20*"=")
    # Find the maximum length of a number in each column
    max_lengths = [max(map(lambda x: len(str(x)), col)) for col in zip(*board)]

    # Create a template string for each row based on the maximum lengths
    template = '  '.join('{{:{}}}'.format(length) for length in max_lengths)

    # Print each row using the template string
    for row in board:
        print(template.format(*row))
    print(20*"=")



In [5]:
def check_win(player:int,board:List[List[int]]) -> bool:
    """
    check if the player has won the game

    Args:
        player (int): player 1 or 2
        board (List[List[int]]): the game board
        
    Returns:
        bool: win or not
    """
    # Check rows
    for i in range(6):
        for j in range(4):
            if board[i][j] == player and board[i][j+1] == player and board[i][j+2] == player and board[i][j+3] == player:
                return True

    # Check columns
    for i in range(3):
        for j in range(7):
            if board[i][j] == player and board[i+1][j] == player and board[i+2][j] == player and board[i+3][j] == player:
                return True

    # Check diagonal (top left to bottom right)
    for i in range(3):
        for j in range(4):
            if board[i][j] == player and board[i+1][j+1] == player and board[i+2][j+2] == player and board[i+3][j+3] == player:
                return True

    # Check diagonal (top right to bottom left)
    for i in range(3):
        for j in range(3, 7):
            if board[i][j] == player and board[i+1][j-1] == player and board[i+2][j-2] == player and board[i+3][j-3] == player:
                return True

    return False

## Simple game with Random players

In [17]:
player = random.choice([1,2])
board = [[0 for j in range(7)] for i in range(6)]
while True:
    #time.sleep(1)
    print()
    print(f"Player {player} plays")
    # Get a random column for the current player to play in
    column = random.randint(0, 6)
    # Find the first empty row in the chosen column
    row = 5
    while row >= 0:
        if board[row][column] == 0:
            break
        row -= 1

    # If the column is full, try again
    if row < 0:
        continue

    # Add the player's piece to the board
    board[row][column] = player

    if check_win(player,board):
        print_board(board)
        print(f"Player {player} wins!")
        break
    # Check if the board is full
    if all(board[i][j] != 0 for i in range(6) for j in range(7)):
        print_board(board)
        print("It's a tie!")
        break
    print_board(board)
    # Switch to the other player
    player = 3 - player  # 3 - player returns the other player (1 -> 2, 2 -> 1)
        


Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  2  0  0  0  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  1  2  0  0  0  0

Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  2  0  0  0  0
0  1  2  0  0  0  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  2  0  0  0  0
0  1  2  1  0  0  0

Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  2  2  0  0  0  0
0  1  2  1  0  0  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  2  2  0  0  0  0
1  1  2  1  0  0  0

Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  2  2  2  0  0  0
1  1  2  1  0  0  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  

### Q learning

Define Q table and hyper parameters learning rate, discount factor and exploration rate (set to one for first games)

In [24]:
Q = {}       # the Q-table
alpha = 0.3  # learning rate
gamma = 0.9  # discount factor
epsilon = 1  # exploration rate

#### Strategy

The main difficulty for training a Q learning algorithm for puissance4 is the number of possible states : $3^{42}$. For Q learning algorithme we need to have a table which maps every state to its Q value. So we need to reduce this number of states. 

One possile way is to train our player only on games with maximum $n_{max}$ step. After $n$ step, if there is no winner, we continue to the next iteration.

Another way, is to aggregate the states : 
- bitboard, only 42 states : the state is the cell where the player put the coin
- number of coins for the player in each row/col ($7^6$ or $6^7$ states)
- number of coins for the player in each row/col ($7^6$ or $6^7$ states)

We can play on the reward to distribute intermediate reward : for example if there is a possible win at the next move -> get a reward

After multiple tests our strategy is the following :
- we train train our player only on games with $n_{max} = 20$ steps
- the state is the board with 1 for the step's player and -1 for the next step's player. So we can can update the Q table at each step
- the two players follow the same strategy 
- The rewards are :
    - 100 if the state is a win
    - $n\_prox*10 - n\_prox\_adv*20 $ where n\_prox is the number of possibilities of winning the game at the next player's move and n\_prox\_adv the number of possibilities of winning the game at the next opponent's move
- we decrease epsilon after some games ranges
- we train on 1 000 000 of games
- we test on 10 000 games of one player following Q table policy vs a random player

In [30]:
def check_prox_win(player:int,board:List[List[int]]) -> int:
    """
    Returns the number of possibilities of winning the game at the next player's move

    Args:
        player (int): the player
        board (List[List[int]]): the game board

    Returns:
        int
    """
    n_prox = 0
    for j in range(7):
        # Check if the column is full
        if board[0][j] != 0:
            continue

        # Simulate the next move in this column
        for i in range(5, -1, -1):
            if board[i][j] == 0:
                board[i][j] = player
                break

        # Check if the player can win in the next move
        if check_win(player,board):
            # Undo the move and return True
            board[i][j] = 0
            n_prox += 1

        # Undo the move
        board[i][j] = 0

    return n_prox

In [31]:
def choose_action(state:Tuple[Tuple[int]],board:List[List[int]])->int:
    """
    choosing a colum given a state and the board following an epsilon greedy strategy

    Args:
        state (Tuple[Tuple[int]]): state of the game
        board (List[List[int]]): the game board

    Returns:
        int: _description_
    """

    # Choose a random action with probability epsilon
    if random.uniform(0, 1) < epsilon:
        column = random.randint(0, 6)
    # Otherwise, choose the action with the highest Q-value
    else:
        max_q_value = -float('inf')
        my_range = list(range(7))
        random.shuffle(my_range)
        for j in my_range:
            if board[0][j] == 0:
                q_value = Q.get(state, {}).get(j, 0)
                if q_value > max_q_value:
                    max_q_value = q_value
                    column = j
    return column



In [32]:
def calculate_reward(player:int,board:List[List[int]])->int:
    """
    Calculate the reward for the player given the board

    Args:
        player (int): the player
        board (List[List[int]]): _description_

    Returns:
        int: reward
    """
    if check_win(player,board):
        return 100
    else:
        n_prox = check_prox_win(player,board)
        n_prox_adv = check_prox_win(3-player,board)
        return n_prox*10   - n_prox_adv*20

### Training

In [33]:
Q = {}       # the Q-table
alpha = 0.3  # learning rate
gamma = 0.9  # discount factor
epsilon = 1  # exploration rate
n_iter = 100000
nb_iter = 0
nb_max_step = 20
# Main game loop
for i in range(n_iter) :
    board = [[0 for j in range(7)] for i in range(6)]
    player = random.choice([1,2])
    if i>=0.9*n_iter:
        epsilon = 0.1
    elif i >=0.8*n_iter:
        epsilon = 0.2
    elif i>=0.7*n_iter:
        epsilon = 0.3
    elif i>= 0.6*n_iter:
        epsilon = 0.5   
    elif i>= 0.5*n_iter:
        epsilon = 0.7
    elif i>=0.4*n_iter:
        epsilon = 0.8
    elif i>= 0.2*n_iter:
        epsilon = 0.9
    nb_iter+=1
    nb_step = 0
    while True and nb_step<nb_max_step:
        nb_step+=1
        lst_view = [[1 if x==player else -1 if x==(3-player) else 0 for x in inner] for inner in board]
        state = tuple(map(tuple, lst_view))
        column = choose_action(state,board)
        # Find the first empty row in the chosen column
        row = 5
        while row >= 0:
            if board[row][column] == 0:
                break
            row -= 1

        # If the column is full, try again
        if row < 0:
            continue

        # Add the player's piece to the board
        board[row][column] = player

        #The new state
        lst_view = [[1 if x==player else -1 if x==(3-player) else 0 for x in inner] for inner in board]
        new_state = tuple(map(tuple, lst_view))

        # Calculate the reward for the current player
        reward = calculate_reward(player,board)

        # Update the Q-value for the previous state-action pair
        
        if state in Q:
            previous_q_value = Q[state].get(column, 0)
        else:
            previous_q_value = 0
            Q[state] = {}
        
        max_q_value = max(Q.get(new_state, {}).get(j, 0) for j in range(7))
        new_q_value = (1 - alpha) * previous_q_value + alpha * (reward + gamma * max_q_value - previous_q_value)
        Q[state][column] = new_q_value

        # Check if the player has won
        if check_win(player,board):
            break
        # Check if the board is full
        if all(board[i][j] != 0 for i in range(6) for j in range(7)):
            break
        # Switch to the other player
        player = 3 - player 
print("Nombre d'états visités différents", len(Q.keys()))

Nombre d'états visités différents 838057


### Testing 

player1 follows the Q table strategy

player2 plays at random

In [39]:
epsilon = 0
n_win = {1:0,2:0}
for i in range(10000):
    board = [[0 for j in range(7)] for i in range(6)]
    player = random.choice([1,2])
    if i==9999:
        print()
        print("LAST PARTY")
    while True :
        if i==9999:
            print()
            print(f"Player {player} plays")
            print_board(board)
        lst_view = [[1 if x==player else -1 if x==(3-player) else 0 for x in inner] for inner in board]
        state = tuple(map(tuple, lst_view))
        if player==1:
            column = choose_action(state,board)
        else : 
            column = random.randint(0, 6)
        #column = choose_action(state)
        # Find the first empty row in the chosen column
        row = 5
        while row >= 0:
            if board[row][column] == 0:
                break
            row -= 1

        # If the column is full, try again
        if row < 0:
            continue

        # Add the player's piece to the board
        board[row][column] = player
        # Check if the player has won
        if check_win(player,board):
            if i==9999:
                print_board(board)
                print(f"Player {player} wins!")
            n_win[player] += 1
            break
        # Check if the board is full
        if all(board[i][j] != 0 for i in range(6) for j in range(7)):
            if i==999:
                print_board(board)
                print("It's a tie!")
            break
        # Switch to the other player
        player = 3 - player  # 3 - player returns the other player (1 -> 2, 2 -> 1)
print(f"WIN rate player 1 : {round(100*(n_win[1]/(n_win[1]+n_win[2])),2)}%")


LAST PARTY

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0

Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
1  0  0  0  0  0  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
1  0  0  0  2  0  0

Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  1  0  0
1  0  0  0  2  0  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  1  0  0
1  0  0  0  2  2  0

Player 2 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
1  0  0  0  1  0  0
1  0  0  0  2  2  0

Player 1 plays
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
0  0  0  0  0  0  0
1  0  0  0  1  0  0
1  0  2  0  2  2  0

Player 2 plays
0  0  0  0  0  0  0


The win rate for our player is close to 50% as we have trained them on only a small number of games compared to the total number of possible states. However, after training the same agents on 1,000,000 games, they were able to achieve an 80% win rate against a random player.

Based on our findings, we believe that a deterministic approach such as the minmax algorithm would outperform our agent and achieve a higher win rate against a random player in the game of Puissance 4. The minmax algorithm selects actions that maximize rewards over the next $n$ steps by exploring all possible options.